# mysql과 python 연동

1. 라이브러리 설치
2. sql 서버의 정보를 등록
3. sql 서버에 접속
4. sql 쿼리문을 이용하여 데이터를 CRUD 작업

In [1]:
# pymysql 설치

!pip install pymysql

     -------------------------------------- 43.7/43.7 kB 712.9 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pymysql

In [3]:
# sql 서버의 정보를 입력
_db = pymysql.connect(
    user = 'root',
    password = '1234',
    host = 'localhost',
    db = 'ubion_77',
    port = 3306
)

In [9]:
# cursor 변수 생성
# sql 쿼리문 실행을 위한 필수 변수
# Dictcursor는 sql 쿼리문이 실행되고 나온 데이터를 딕셔너리 형태로 받는다.

cursor = _db.cursor(pymysql.cursors.DictCursor)

In [10]:
sql = """
select * from user_info  
"""

cursor.execute(sql)

2

In [11]:
result = cursor.fetchall()

In [8]:
import pandas as pd

In [12]:
pd.DataFrame(result)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01023432322,1


In [13]:
sql = """
select * from example
"""

cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df

,Name,Gender,Phone
0,A,male,010-1234-5678
1,B,female,010-1234-5679
2,C,male,010-1234-5680
3,D,female,010-1234-5681
4,E,male,010-1234-5682
5,F,female,010-1234-5683
6,G,male,010-1234-5684
7,H,female,010-1234-5685
8,I,male,010-1234-5686
9,J,female,010-1234-5687


In [16]:
sql = """
insert into
user_info
values
("test3", "1234", "test3","01023454321",2)

"""

cursor.execute(sql)


1

In [17]:
# commit 까지 써줘야 sql에 수정 완료
_db.commit()

### sql 함수화
1. 첫번째 함수
    - 매개변수 1개(sql 쿼리문)
    - sql 쿼리문을 실행하고 결과를 데이터프레임으로 변경하여 리턴

2. 두번째 함수
    - 매개변수 1개 (sql쿼리문)
    - sql 쿼리문을 실행하고 데이터베이스를 변경

3. 세 번째 함수 
    - 1번 함수와 2번 함수를 하나의 함수에서 실행이 될 수 있도록 생성 

In [50]:
sql = """
select * from user_info
"""

def func_1 (sql):
    
    # 쿼리문 실행
    cursor.execute(sql)
    # 쿼리문 결과
    result = cursor.fetchall()
    return pd.DataFrame(result)
func_1(sql)    

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01023432322,1
2,test21,1234,test21,01023722455,1
3,test3,1234,test3,01023454321,2
4,test6,1234,test6,01025232342,2


In [26]:
def func_2 (sql, values):
    cursor.execute(sql,values)
    _db.commit()


In [25]:
sql = """
insert into
user_info
values
(%s,%s,%s,%s,%s)
"""
values = ["test10", "1232", "test10", "01023422455", 1]
func_2(sql,values)

In [29]:
def func_3(sql, values = []):
    if (sql.find('select') != -1) and (sql.find('select') < 10):
        cursor.execute(sql)
        result = cursor.fetchall()
        result = pd.DataFrame(result)
    else :
        cursor.execute(sql,values)
        _db.commit()
        result = "Query Ok"
        return result
        

In [30]:
sql = """
insert into
user_info
values
(%s,%s,%s,%s,%s)
"""
values = ["test21", "1234", "test21", "01023722455", 1]
func_3(sql,values)

'Query Ok'

In [32]:
sql = """
delete
from
user_info
where
user_id = %s
"""
values = ['test10']
func_3(sql, values)

'Query Ok'

In [168]:
import imp
imp.reload(mod_sql)

<module 'mod_sql' from 'c:\\Users\\user\\Desktop\\욱\\230417\\mod_sql\\__init__.py'>

In [89]:
import mod_sql

In [169]:
sql_d = mod_sql.Database('localhost',
    'root',
    '1234',
    'ubion7',
    3306)

In [113]:
sql = """
select * from user_info
"""
sql_d.func_3(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01023432322,1
2,test3,1234,test3,01023454321,2
3,test6,1234,test6,01025232342,2


In [65]:
test2 = mod_sql.Database(
    '172.16.11.148',
    'ubion',
    '1234',
    'ubion7',
    3306
)

In [72]:
sql = """
insert into
user_info
values
(%s,%s,%s,%s,%s)
"""
values = ["test21", "1234", "test21", "01023722455", 1]
test2.func_3(sql,values)

'Query Ok'

In [ ]:
sql = """
select
Name as '이름',
Phone as '휴대폰번호'
from
example
where
Gender = '%s'
"""
values = ['female']
sql_d.func_3(sql,values)

In [124]:
sql = """
select
`Name` as '이름',
`Phone` as '휴대폰번호'
from
example
where
Gender = %s
limit 3
"""
values = ['female']
sql_d.func_3(sql, values)

,이름,휴대폰번호
0,B,010-1234-5679
1,D,010-1234-5681
2,F,010-1234-5683


In [134]:
sql = """
select
createDt as '등록일시',
deathCnt as '사망자',
decideCnt as '감염자'
from
corona
""" 

sql_d.func_3(sql)


,등록일시,사망자,감염자
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


In [130]:
import pandas as pd

In [140]:
df = pd.read_csv('../230331/230403/230404/csv/corona.csv')
df

,Unnamed: 0,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,0,2022-06-08 09:09:05.982,24305,18188200,904,20220608,00:00,NaN,NaN,NaN
1,1,2022-06-07 09:09:00.897,24299,18174842,903,20220607,00:00,2022-06-08 09:10:36.846,NaN,NaN
2,2,2022-06-06 09:00:06.734,24279,18168670,902,20220606,00:00,2022-06-08 09:10:50.441,NaN,NaN
3,3,2022-06-05 08:53:19.426,24258,18163648,901,20220605,00:00,2022-06-08 09:11:04.758,NaN,NaN
4,4,2022-06-04 08:56:49.219,24238,18153814,900,20220604,00:00,2022-06-08 09:11:26.303,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
815,815,2020-03-14 00:00:00.000,72,8086,55,20200314,00:00,2021-10-07 10:30:51.51,261335.0,3.318000
816,816,2020-03-13 00:00:00.000,67,7979,54,20200313,00:00,2021-10-07 10:30:51.51,248647.0,3.458499
817,817,2020-03-12 00:00:00.000,66,7869,53,20200312,00:00,2021-10-07 10:30:51.51,234998.0,3.621744
818,818,2020-03-11 00:00:00.000,60,7755,52,20200311,00:00,2021-10-07 10:30:51.51,222395.0,3.804175


In [150]:
df = df.loc[:,['createDt','deathCnt','decideCnt']]
df

,createDt,deathCnt,decideCnt
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


In [164]:
df.columns = ['등록일시', '사망자', '감염자']
df

,등록일시,사망자,감염자
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


In [170]:
sql = """ 
select
`Country` as '국가',
`Units Sold` as '판매개수',
`Unit Price` as '가격',
`Unit Cost` as '원가',
(`Units Sold` * `Unit Price`) as '총수익',
(`Units Sold` * `Unit Cost`) as '총비용',
(`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
from 
`Sales Records`
"""
sql_d.func_3(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20
